In [2]:
from app.onnx_inference import BrioOnnxPipeline
from transformers import PegasusTokenizer, BartTokenizer, PegasusForConditionalGeneration, BartForConditionalGeneration
import time

pegasus_model_paths = ['./models/brio-xsum-cased-encoder-quantized.onnx',
                       './models/brio-xsum-cased-decoder-quantized.onnx', 
                       './models/brio-xsum-cased-init-decoder-quantized.onnx']

bart_model_paths = ['./models/brio-cnndm-uncased-encoder-quantized.onnx',
                    './models/brio-cnndm-uncased-decoder-quantized.onnx',
                    './models/brio-cnndm-uncased-init-decoder-quantized.onnx']

pegasus_checkpoint = 'Yale-LILY/brio-xsum-cased'
bart_checkpoint = 'Yale-LILY/brio-cnndm-uncased'

text= """
The Economist is a British weekly newspaper printed in demitab format and published digitally that focuses on current affairs, international business, politics, technology, and culture. Based in London, the newspaper is owned by The Economist Group, with core editorial offices in the United States, as well as across major cities in continental Europe, Asia, and the Middle East. In 2019, its average global print circulation was over 909,476; this, combined with its digital presence, runs to over 1.6 million. Across its social media platforms, it reaches an audience of 35 million, as of 2016. The newspaper has a prominent focus on data journalism and analysis over original reporting, to both criticism and acclaim. Founded in 1843, The Economist was first circulated by Scottish economist James Wilson to muster support for abolishing the British Corn Laws (1815–1846), a system of import tariffs. Over time, the newspaper's coverage expanded further into political economy and eventually began running articles on current events, finance, commerce, and British politics. Throughout the mid-to-late 20th century, it greatly expanded its layout and format, adding opinion columns, special reports, political cartoons, reader letters, cover stories, art critique, book reviews, and technology features. The paper is often recognizable by its fire engine red nameplate and illustrated, topical covers. Individual articles are written anonymously, with no byline, in order for the paper to speak as one collective voice. It is supplemented by its sister lifestyle magazine, 1843, and a variety of podcasts, films, and books.
"""
print('Loading Pegasus Models')
pegasus_pipeline = BrioOnnxPipeline(pegasus_checkpoint, pegasus_model_paths)
pegasus_model = PegasusForConditionalGeneration.from_pretrained(pegasus_checkpoint)
pegasus_tokenizer = PegasusTokenizer.from_pretrained(pegasus_checkpoint)
print('Loaded')

print('Loading Bart Models')
bart_pipeline = BrioOnnxPipeline(bart_checkpoint, bart_model_paths, pegasus=False)
bart_model = BartForConditionalGeneration.from_pretrained(bart_checkpoint)
bart_tokenizer = BartTokenizer.from_pretrained(bart_checkpoint)
print('Loaded')
print('-'*10)

start = time.time()
summary = pegasus_pipeline(text)
end = time.time()
print(f"Brio-Pegasus ONNX : {end-start:.4f}")
print(summary)
print('-'*10)

start = time.time()
encoded_input = pegasus_tokenizer(text,return_tensors='pt')
output_tokens = pegasus_model.generate(**encoded_input)
summary = pegasus_tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]
end = time.time()
print(f"Brio-Pegasus Vanilla : {end-start:.4f}")
print(summary)
print('-'*10)

start = time.time()
summary = bart_pipeline(text)
end = time.time()
print(f"Brio-Bart ONNX : {end-start:.4f}")
print(summary)
print('-'*10)

start = time.time()
encoded_input = bart_tokenizer(text,return_tensors='pt')
output_tokens = bart_model.generate(**encoded_input)
summary = bart_tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]
end = time.time()
print(f"Brio-Bart Vanilla : {end-start:.4f}")
print(summary)


Loading Pegasus Models
Creating ONNX sessions...
ONNX sessions created!
Loading PEGASUS model...
PEGASUS model loaded!
Loading PEGASUS tokenizer...
PEGASUS tokenizer loaded!
Loaded
Loading Bart Models
Creating ONNX sessions...
ONNX sessions created!
Loading BART model...
BART model loaded!
Loading BART tokenizer...
BART tokenizer loaded!
Loaded
----------
Brio-Pegasus ONNX : 10.4121
The Economist is one of the world's leading business and political newspapers.
----------
Brio-Pegasus Vanilla : 17.0237
The Economist is one of the world's leading business and political newspapers, with a strong focus on data journalism and analysis.
----------
Brio-Bart ONNX : 5.1177
The Economist is a weekly newspaper printed in demitab format and published digitally. Founded in 1843, it focuses on current affairs, international business, politics, and culture. The newspaper has a global circulation of over 1.6 million. The Economist is owned by The Economist Group.
----------
Brio-Bart Vanilla : 13.297